# ИД23-1 Маслов АН
## Пример 1. Программа создает случайное количество строк в 4 процессах, объединяет их и записывает в csv файл

In [15]:
%%file rand_csv.py
import random
import time

def rand_csv(length, time_start, time_end, output): #генератор случайно даты и числа
    csv_rows = []
    for i in range(length):
        stime = time.mktime(time.strptime(time_start, '%m/%d/%Y %I:%M %p'))
        etime = time.mktime(time.strptime(time_end, '%m/%d/%Y %I:%M %p'))
        ptime = stime + random.random() * (etime - stime)
        csv_rows.append([time.strftime('%y%m%d%H%M%S', time.localtime(ptime)), random.randint(1, 9999999)])
    output.put(csv_rows)

Overwriting rand_csv.py


In [16]:
import rand_csv

In [17]:
import multiprocessing as mp
import random
import pandas as pd

output = mp.Queue()

processes = [mp.Process(target=rand_csv.rand_csv, args=(random.randint(50000,500000), "1/1/2008 1:30 PM", "1/1/2025 4:50 AM", output)) \
             for x in range(4)] #создаем 4 процесса
for p in processes:
    p.start()

results = [output.get() for p in processes] #сохраняем результат

for p in processes:
    p.join()

pandas_result = []
for i in results:
    pandas_result = pandas_result + i #форматируем для удобства

resultdata = pd.DataFrame(pandas_result, columns=["datetime", "rand_number"]) #создаем датафрейм
resultdata['datetime'] = pd.to_datetime(resultdata['datetime'], format='%y%m%d%H%M%S')
resultdata = resultdata.sort_values('datetime') #сортируем по дате
resultdata.to_csv("csv_result.csv", sep=';', index=False) #записываем результат в csv файл
resultdata


,datetime,rand_number
451096,2008-01-01 13:42:09,6030757
549561,2008-01-01 13:56:57,6484828
268314,2008-01-01 14:21:55,9842722
1086548,2008-01-01 14:26:49,4251113
154830,2008-01-01 14:28:02,2809513
...,...,...
1262015,2025-01-01 04:06:14,1625747
1075113,2025-01-01 04:19:37,9350572
89371,2025-01-01 04:21:04,4600696
396669,2025-01-01 04:36:52,7457410


## Пример 2. Программа формирует две очереди, одну и задачами и вторую с результатом этих задач

In [18]:
%%file worker_time.py
import time

def mul_task(x): # Возведение в квадрат с задержкой
    time.sleep(0.5)
    return x * x

def worker_time(task_queue, result_queue): #Берет задачи из очерещди и результат кладет в другую очередь
    while True:
        x = task_queue.get()
        if x is None:   # сигнал остановки
            break
        result = mul_task(x)
        result_queue.put(result)

Overwriting worker_time.py


In [19]:
import worker_time

In [20]:
import multiprocessing as mp

num_workers = 4
task_queue = mp.Queue()
result_queue = mp.Queue()

# Запускаем рабочих
workers = []
for w in range(num_workers):
    p = mp.Process(target=worker_time.worker_time, args=(task_queue, result_queue))
    p.start()
    workers.append(p)

# Кладём задачи
for i in range(10):
    task_queue.put(i)

# Отправляем сигнал остановки
for w in range(num_workers):
    task_queue.put(None)

# Сбор результатов
results = []
for w in range(10):
    results.append(result_queue.get())

# Ожидание завершения
for p in workers:
    p.join()

print("Результаты:", results)

Результаты: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


## Пример 3. Программа формирует список дат и дня недели и упорядочивает их

In [21]:
%%file process_date.py
from datetime import datetime, timedelta
import time
def process_date(date_str): # Функция обработки даты
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    time.sleep(0.3)  # имитация работы
    return date_str, date_obj.strftime("%A")  # (дата, день недели)

Overwriting process_date.py


In [22]:
import process_date

In [23]:
from multiprocessing import Pool
from datetime import datetime, timedelta
import time



# Формируем список дат
start_date = datetime(2025, 1, 1)
dates = [(start_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(10)]

with Pool(processes=4) as pool:
    # Пример с map
    ordered_results = pool.map(process_date.process_date, dates)

    # Пример с apply
    manual_results = []
    for d in dates:
        manual_results.append(pool.apply(process_date.process_date, (d,)))

print("Результаты (map):")
for d, day in ordered_results:
    print(f"{d} - {day}")

print("\nРезультаты (apply):")
for d, day in manual_results:
    print(f"{d} - {day}")


Результаты (map):
2025-01-01 - Wednesday
2025-01-02 - Thursday
2025-01-03 - Friday
2025-01-04 - Saturday
2025-01-05 - Sunday
2025-01-06 - Monday
2025-01-07 - Tuesday
2025-01-08 - Wednesday
2025-01-09 - Thursday
2025-01-10 - Friday

Результаты (apply):
2025-01-01 - Wednesday
2025-01-02 - Thursday
2025-01-03 - Friday
2025-01-04 - Saturday
2025-01-05 - Sunday
2025-01-06 - Monday
2025-01-07 - Tuesday
2025-01-08 - Wednesday
2025-01-09 - Thursday
2025-01-10 - Friday
